In [1]:
from abc import ABC, abstractmethod
from collections import Counter
from itertools import combinations

In [2]:
class Utils:
    def check_counts(self, string):
        for item, value in Counter(string).items():
            if value % 2 == 1:
                return False
        return True

    def generate_substrings(self, string, masterlist):
        if string not in masterlist:
            if self.check_counts(string):
                masterlist.add(string)
        if len(string) < 3:
            return
        else:
            self.generate_substrings(string[1:], masterlist)
            self.generate_substrings(string[:-1], masterlist)

    def evaluate_combination(self, a, b, substring):
        for i in range(len(a)):
            if substring[a[i]] != substring[b[i]]:
                return False
        return True

    def check_substring_for_twins(self, substring):
        look_further = False
        length = len(substring)
        indices = set(range(length))

        for a in combinations(indices, int(length / 2)):
            b = tuple(indices - set(a))
            if self.evaluate_combination(a, b, substring):
                print(f"Found twins in {substring} with indices {a} and {b}")
                return True
        return False

In [3]:
class GameState:
    
    def __init__(self, config):
        self.config = config
        self.current_string = ""
        self.current_position = 0
        self.alphabet = self.generate_alphabet(config['m'])
        self.winner = None
        self.playing = True
        #self.setup_config(config)
        #self.setup_initial_values(self)
    
    def setup_config(self, config):
        self.config = config
    
    def setup_initial_values(self, config):
        self.current_string = ""
        self.current_position = 0
        self.alphabet = self.generate_alphabet(config['m'])
        self.winner = None
        self.playing = True
    
    def get_current_string(self):
        return self.current_string
    
    def get_alphabet(self):
        return self.alphabet
    
    def get_current_position(self):
        return self.current_position
    
    def get_winner(self):
        return self.winner
    
    def get_string_length(self):
        return len(self.current_string)
    
    def set_current_position(self, position):
        self.current_position = position
    
    def set_alphabet(self, alphabet):
        self.alphabet = alphabet
    
    def set_winner(self, player):
        self.winner = player
        
    def set_current_string(self, string):
        self.current_string = string
    
    def is_playing(self):
        return self.playing
    
    def is_not_playing(self):
        self.playing = False
    
    def restart_game(self):
        self.setup_initial_values(self)
    
    def update_current_string(self, char):
        self.current_string = self.current_string[:self.current_position] + char + self.current_string[self.current_position:]
    
    def generate_alphabet(self, number_of_letters_in_alphabet):
        return ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'][0:number_of_letters_in_alphabet + 1]
    
    def get_number_of_letters_from_user():
        number_of_letters_in_alphabet = None
        while number_of_letters_in_alphabet == None:
            try:
                number_of_letters_in_alphabet = int(input("How many letters in alphabet: "))
                if number_of_letters_in_alphabet < 1 or number_of_letters_in_alphabet > 26:
                    number_of_letters_in_alphabet = None
                    print("Number of letters in alphabet should be more than 0 and less than 26")
            except:
                number_of_letters_in_alphabet = None
                print("Invalid input!")
        
        return number_of_letters_in_alphabet
    
    def get_max_word_length_from_user():
        max_word_length = None
        while max_word_length == None:
            try:
                max_word_length = int(input("How maximal length word should have: "))
                if max_word_length < 1:
                    max_word_length = None
                    print("Maximal word length should be more than 0")
            except:
                max_word_length = None
                print("Invalid input!")
        
        return max_word_length

In [4]:
class Player(ABC):
    
    @abstractmethod
    def move(self):
        pass
    
class Human(Player):
    def move(self, game_state):
        self.print_possible_positions(game_state)

        position = None
        while position == None:
            try:
                position = int(input("Choose position: "))
                if not self.check_validity(position, game_state):
                    position = None
                    print("Invalid position!")
            except:
                position = None
                print("Invalid input!")
            
        game_state.set_current_position(position)
        
    def check_validity(self, position, game_state):
        current_string = game_state.get_current_string()
        return position >= 0 and position <= len(current_string)
    
    def print_possible_positions(self, game_state):
        current_string = game_state.get_current_string()
        print(current_string)
        print_string = "_"
        for letter in current_string:
            print_string += letter
            print_string += "_"
        
        number_string = ""
        num = 0
        for character in print_string:
            if character == "_":
                number_string += str(num)
                num += 1
            else:
                number_string += " "
        print(f"{print_string}\n{number_string}")
        
class AI(Player):
    def move(self, game_state):
        alphabet = game_state.get_alphabet()
        current_string = game_state.get_current_string()
        current_position = game_state.get_current_position()
        
        char = self.find_best_letter(current_string, current_position, alphabet)
        game_state.update_current_string(char)
        
    def evaluate_single_string(self, string):
        return self.check_for_twins(string)
    
    def check_for_twins(self, string):
        utils = Utils()
        substrings = set()
        utils.generate_substrings(string, substrings)
        for substring in substrings:
            if utils.check_substring_for_twins(substring):
                return True
        return False
    
    def generate_strings(self, current_string, current_position, alhpabet):
        beggining = current_string[:current_position]
        end = current_string[current_position:]
        strings = [f"{beggining}{char}{end}" for char in alhpabet]
        
        return strings
    
    def find_best_letter(self, current_string, current_position, alphabet):
        candidates = self.generate_strings(current_string, current_position, alphabet)
        
        for candidate in candidates:
            #print(candidate)
            if not self.evaluate_single_string(candidate):
                return candidate[current_position]
        print('No good options')
        return alphabet[0]

    

In [5]:
class Game:
    def __init__(self, config):
        self.player1 = Human()
        self.player2 = AI()
        self.config = config
        self.game_state = GameState(config)
        
    def play(self):
        while(self.game_state.is_playing()):
            self.game_state.set_winner(None)
            #self.game_state.set_current_string('abca')
            while(self.game_state.get_string_length() < self.config['n']):
                print('Player 1:')
                self.player1.move(self.game_state)
                print('Player 2:')
                self.player2.move(self.game_state)
                if self.evaluate_string(self.game_state.get_current_string()):
                    self.game_state.set_winner('Player1')
                    break
            if not self.game_state.get_winner():
                self.game_state.set_winner('Player2')
            print(f"{self.game_state.get_winner()} wins!")
            
            play_again_user_answer = input("Play again? y/n: ")
            if play_again_user_answer == "n":
                self.game_state.is_not_playing()
            elif play_again_user_answer == "y":
                self.game_state.restart_game()
                
    def check_for_twins(self, string):
        utils = Utils()
        substrings = set()
        utils.generate_substrings(string, substrings)
        for substring in substrings:
            if utils.check_substring_for_twins(substring):
                return True
        return False
                
    def evaluate_string(self, string):
        return self.check_for_twins(string)
                

In [6]:
number_of_letters_in_alphabet = GameState.get_number_of_letters_from_user()
max_word_length = GameState.get_max_word_length_from_user()

How many letters in alphabet:  3
How maximal length word should have:  3


In [7]:
simple_config = {'m': number_of_letters_in_alphabet, 'n': max_word_length}

In [8]:
game = Game(simple_config)

In [9]:
game.play()

Player 1:

_
0


Choose position:  0


Player 2:
Player 1:
a
_a_
0 1


Choose position:  0


Player 2:
Found twins in aa with indices (0,) and (1,)
Player 1:
ba
_b_a_
0 1 2


Choose position:  0


Player 2:
Player2 wins!


Play again? y/n:  y


TypeError: 'GameState' object is not subscriptable